## 1. About

This noteboook takes in as input the data gotten from the [SPICE](https://en.wikipedia.org/wiki/SPICE) simulation about the circuit and reformats them into a format more usable.

### 1.1 About this New Version

The old notebook was laggy and not well documented, this notebook aims to improve this. 

Mainly by exchanging the **pandas** libary through **polars** libary.

### 1.2 Explanation of the Data

### 1.3 General

Learnable SG Data

#### Dataset 1

Inputs:  
1. Processed_VIN_of_Constant_1st_Neuron_Vin0-1.csv
2. Processed_VIN_of_Constant_1st_Neuron_Vin1-2.csv

Outputs:  
1. Processed_output_Vout_of_Constant_1st_Neuron_Vin0-1.csv
2. Processed_output_Vout_of_Constant_1st_Neuron_Vin1-2.csv
3. Processed_output_Vout_of_Constant_2nd_Neuron_Vin0-1.csv
4. Processed_output_Vout_of_Constant_2nd_Neuron_Vin1-2.csv

Parameters:     
- a. C1 (10u-100u)
- b. CIN (10u-100u)
- c. COUT (10u-100u)
- d. L (40u-100u)
- e. R1 (20k-50k)
- f. W1 (600u-1000u)
- g. VIN (input voltage) (0:1, 1:2)

#### Dataset 2

Inputs:  
1. Processed_Vin_sin1.csv
2. Processed_Vin_sin2.csv
3. Processed_Vin_sin3.csv

Outputs:  
1. Processed_output_Vout_of_Sin1.csv
2. Processed_output_Vout_of_Sin2.csv
3. Processed_output_Vout_of_Sin3.csv

Parameters:    
- a. C1 (10u-100u)
- b. CIN (10u-100u)
- c. COUT (10u-100u)
- d. L (40u-100u)
- e. R1 (20k-50k)
- f. W1 (600u-1000u)
- g. a  (inputs to create sin wave of multiple freq) (0:0.25, 0.25:0.5,0.5:1)
- h. b (0:180,180:360,180:360)
- i. f (1:3,3:5,3:5)
  

In [1]:
# Number of Entries of the dataset we want to use
# For Testing purpose to generate all the data set to "None"
NUMBER_DATASET_LENGTH = None

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import re
import warnings
import polars as pl

## 2. Processing Inputs of Dataset 1

We have input voltage data (VIN) in df_input_1 and df_input_2.
And output voltage data (VOUT) in df_output_2 and df_output_2.

Each CSV is organized like this:

```csv
X (time?)	Column 1	Column 2	Column 3	…
0.00	0.04193	0.04193	0.04193	…
0.01	0.04193	0.04193	0.04193	…
```

Columns are named with parameter information inside parentheses, e.g.:

```
/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.111111,w1=0.000776)
```

Each column corresponds to a simulation run with a particular set of circuit parameters and input voltage and each row entry represents one timestep, whereby the entry the current input voltage of the time step is.

In [3]:
DATA_LOCATION = "data/V2/"

In [4]:
df_input_1 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_VIN_of_Constant_1st_Neuron_Vin0-1_v2.csv"))
df_input_2 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_VIN_of_Constant_1st_Neuron_Vin1-2_v2.csv"))
df_input_1.head()

X,"/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.2222222,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.3333333,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.4444444,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.5555556,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.6666667,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.7777778,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.8888889,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1,w1=0.000776) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.1111111,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.2222222,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.3333333,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.4444444,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.5555556,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.6666667,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.7777778,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.8888889,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1,w1=0.000734) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.1111111,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.2222222,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.3333333,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.4444444,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.5555556,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.6666667,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.7777778,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.8888889,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1,w1=0.00065) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.1111111,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.2222222,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.3333333,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.4444444,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.5555556,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.6666667,w1=0.000999) Y",…,"/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.3333333,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.4444444,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.5555556,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.6666667,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.7777778,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.8888889,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1,w1=0.000918) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3e-05,R1=46163,VIN=0,w1=0.000772) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3

In [5]:
df_output_1 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_1st_Neuron_Vin0-1_v2.csv"))
df_output_2 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_1st_Neuron_Vin1-2_v2.csv"))
df_output_2.head(50)

X,"/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.111111,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.222222,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.333333,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.444444,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.555556,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.666667,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.777778,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1.888889,w1=0.000776) Y","/Vout_buffer2 (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=2,w1=0.000776) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.111111,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.222222,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.333333,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.444444,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.555556,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.666667,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.777778,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1.888889,w1=0.000734) Y","/Vout_buffer2 (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=2,w1=0.000734) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.111111,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.222222,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.333333,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.444444,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.555556,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.666667,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.777778,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1.888889,w1=0.00065) Y","/Vout_buffer2 (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=2,w1=0.00065) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.111111,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.222222,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.333333,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.444444,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.555556,w1=0.000999) Y","/Vout_buffer2 (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=1.666667,w1=0.000999) Y",…,"/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.333333,w1=0.000918) Y","/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.444444,w1=0.000918) Y","/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.555556,w1=0.000918) Y","/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.666667,w1=0.000918) Y","/Vout_buffer2 (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=3

### 2.1 Goal of Expand Function

The goal with this function is to make is so that the paramaeters of the simulation run that are embedded in the column name are instead added to the tail of the dataframe.
This means afetr using the expand fucntion, each column in the dataframe is one of the simulation run, where each entry is the voltage at a particular time, while at the end of the column the parameetrs are added, for eahc parameetr one new row. 

In [6]:
def expand(df, params=None, id_col="X"):
    if params is None:
        params = ["C1", "CIN", "COUT", "L", "R1", "w1"]

    cols = [c for c in df.columns if c != id_col]

    def parse_col(colname: str):
        try:
            inside = colname.split("(", 1)[1].split(")", 1)[0]
        except Exception:
            return {}
        parsed = {}
        for part in inside.split(","):
            if "=" not in part:
                continue
            k, v = part.split("=", 1)
            try:
                parsed[k.strip()] = float(v.strip())
            except Exception:
                parsed[k.strip()] = None
        return parsed

    parsed = [parse_col(c) for c in cols]

    rows = []
    for p in params:
        row = {id_col: p}
        for i, c in enumerate(cols):
            row[c] = parsed[i].get(p)
        rows.append(row)
    params_df = pl.DataFrame(rows)

    # concat with relaxed typing
    combined = pl.concat([df.with_columns(pl.col(id_col).cast(pl.Utf8)), params_df],
                         how="vertical_relaxed")

    # overwrite the id_col with sequential labels like in pandas
    labels = [str(i) for i in range(len(df))] + params  # 0..N-1, then C1, ...
    combined = combined.with_columns(pl.Series(id_col, labels))

    return combined

In [7]:
%%time

df_input_1_combined = expand(df_input_1[:NUMBER_DATASET_LENGTH])
df_input_2_combined = expand(df_input_2[:NUMBER_DATASET_LENGTH])

CPU times: user 12.3 s, sys: 1.72 s, total: 14 s
Wall time: 11.5 s


In [8]:
df_input_1_combined.head()

X,"/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.2222222,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.3333333,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.4444444,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.5555556,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.6666667,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.7777778,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.8888889,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1,w1=0.000776) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.1111111,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.2222222,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.3333333,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.4444444,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.5555556,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.6666667,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.7777778,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.8888889,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1,w1=0.000734) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.1111111,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.2222222,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.3333333,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.4444444,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.5555556,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.6666667,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.7777778,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.8888889,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1,w1=0.00065) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.1111111,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.2222222,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.3333333,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.4444444,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.5555556,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.6666667,w1=0.000999) Y",…,"/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.3333333,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.4444444,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.5555556,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.6666667,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.7777778,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=0.8888889,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1,w1=0.000918) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3e-05,R1=46163,VIN=0,w1=0.000772) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3

In [9]:
df_input_1_combined.tail(50)[:, 0:2]

X,"/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0,w1=0.000776) Y"
str,f64
"""1457""",0.0
"""1458""",0.0
"""1459""",0.0
"""1460""",0.0
"""1461""",0.0
…,…
"""CIN""",0.00007
"""COUT""",0.000035
"""L""",0.000066


Then concatenate them side by side, so your final input DataFrame dfin_const contains:
- All signal traces from dfin1 and dfin2
- All the extracted circuit parameters repeated for each corresponding trace  

In [10]:
def safe_concat(dfs, how="horizontal"):
    """
    Concatenate Polars DataFrames horizontally,
    automatically renaming duplicate columns (X, X_2, X_3, ...).
    """
    seen = {}
    renamed = []
    
    for df in dfs:
        new_cols = []
        for c in df.columns:
            if c not in seen:
                seen[c] = 1
                new_cols.append(c)
            else:
                seen[c] += 1
                new_cols.append(f"{c}_{seen[c]}")
        renamed.append(df.rename(dict(zip(df.columns, new_cols))))
    
    return pl.concat(renamed, how=how)

In [11]:
%%time
df_input_dataset_1_concat = safe_concat([df_input_1_combined, df_input_2_combined])

df_input_dataset_1_concat.tail(8)

CPU times: user 422 ms, sys: 26.6 ms, total: 448 ms
Wall time: 449 ms


X,"/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.2222222,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.3333333,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.4444444,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.5555556,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.6666667,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.7777778,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=0.8888889,w1=0.000776) Y","/VIN (C1=9.4e-05,CIN=7e-05,COUT=3.5e-05,L=6.6e-05,R1=20473,VIN=1,w1=0.000776) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.1111111,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.2222222,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.3333333,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.4444444,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.5555556,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.6666667,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.7777778,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=0.8888889,w1=0.000734) Y","/VIN (C1=3.8e-05,CIN=7.7e-05,COUT=6.6e-05,L=7.8e-05,R1=22511,VIN=1,w1=0.000734) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.1111111,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.2222222,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.3333333,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.4444444,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.5555556,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.6666667,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.7777778,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=0.8888889,w1=0.00065) Y","/VIN (C1=8.4e-05,CIN=5.8e-05,COUT=5.8e-05,L=5.5e-05,R1=37296,VIN=1,w1=0.00065) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.1111111,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.2222222,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.3333333,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.4444444,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.5555556,w1=0.000999) Y","/VIN (C1=5.1e-05,CIN=4.9e-05,COUT=4.7e-05,L=6.4e-05,R1=20139,VIN=0.6666667,w1=0.000999) Y",…,"/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.333333,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.444444,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.555556,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.666667,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.777778,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=1.888889,w1=0.000918) Y","/VIN (C1=5.1e-05,CIN=7.5e-05,COUT=9.8e-05,L=5.6e-05,R1=30871,VIN=2,w1=0.000918) Y","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3e-05,R1=46163,VIN=1,w1=0.000772) Y_2","/VIN (C1=5.7e-05,CIN=8.9e-05,COUT=1.9e-05,L=4.3e-05

## 3. Processing Outputs of Dataset 1

Now we do the same for the output datasets.

In [12]:
df_output_1 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_1st_Neuron_Vin0-1_v2.csv"))
df_output_2 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_1st_Neuron_Vin1-2_v2.csv"))

In [13]:
df_output_3 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_2nd_Neuron_Vin0-1_v2.csv"))
df_output_4 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Constant_2nd_Neuron_Vin1-2_v2.csv"))

In [14]:
%%time

df_output_1_combined = expand(df_output_1[:NUMBER_DATASET_LENGTH])
df_output_2_combined = expand(df_output_2[:NUMBER_DATASET_LENGTH])
df_output_3_combined = expand(df_output_1[:NUMBER_DATASET_LENGTH])
df_output_4_combined = expand(df_output_2[:NUMBER_DATASET_LENGTH])

CPU times: user 10.4 s, sys: 1.08 s, total: 11.5 s
Wall time: 11.2 s


In [15]:
%%time
df_output_1st_neuron_dataset_1_concat = safe_concat([df_output_1_combined, df_output_2_combined])
df_output_2nd_neuron_dataset_1_concat = safe_concat([df_output_3_combined, df_output_4_combined])

CPU times: user 651 ms, sys: 21.1 ms, total: 672 ms
Wall time: 650 ms


## 4. Processing of Dataset 2

In [16]:
df_input_1 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_sin1_v2.csv"))
df_input_2 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_sin2_v2.csv"))
df_input_3 = pl.read_csv(str(DATA_LOCATION + "Input/" + "Processed_sin3_v2.csv"))

In [17]:
df_output_1 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Sin1_v2.csv"))
df_output_2 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Sin2_v2.csv"))
df_output_3 = pl.read_csv(str(DATA_LOCATION + "/Output/" + "Processed_output_Vout_of_Sin3_v2.csv"))

In [18]:
df_input_1_combined = expand(df_input_1[:NUMBER_DATASET_LENGTH])
df_input_2_combined = expand(df_input_2[:NUMBER_DATASET_LENGTH])
df_input_3_combined = expand(df_input_3[:NUMBER_DATASET_LENGTH])

df_input_dataset_2_concat = safe_concat([df_input_1_combined, df_input_2_combined, df_input_3_combined])

In [19]:
df_output_1_combined = expand(df_output_1[:NUMBER_DATASET_LENGTH])
df_output_2_combined = expand(df_output_2[:NUMBER_DATASET_LENGTH])
df_output_3_combined = expand(df_output_1[:NUMBER_DATASET_LENGTH])

df_output_dataset_2_concat = safe_concat([df_output_1_combined, df_output_2_combined, df_output_3_combined])

## 5. Combining the Datasets into One

In [20]:
# We drop the index and tranform it into a pytorch tensor 

VIN = torch.cat([
    torch.tensor(df_input_dataset_1_concat.drop("X", "X_2").to_numpy().astype("float64")),
    torch.tensor(df_output_1st_neuron_dataset_1_concat.drop("X", "X_2").to_numpy().astype("float64")),
    torch.tensor(df_input_dataset_2_concat.drop("X", "X_2", "X_3").to_numpy().astype("float64"))
], 1)

VOUT = torch.cat([
    torch.tensor(df_output_1st_neuron_dataset_1_concat.drop("X", "X_2").to_numpy().astype("float64")),
    torch.tensor(df_output_2nd_neuron_dataset_1_concat.drop("X", "X_2").to_numpy().astype("float64")),
    torch.tensor(df_output_dataset_2_concat.drop("X", "X_2", "X_3").to_numpy().astype("float64"))
], 1)

VIN.shape, VOUT.shape

(torch.Size([1507, 63993]), torch.Size([1507, 63993]))

In [21]:
X = torch.tensor(VIN.clone().detach()).float().T.unsqueeze(2)
Y = torch.tensor(VOUT.clone().detach()).float().T

X.shape, Y.shape

/tmp/ipykernel_25497/2295200637.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(VIN.clone().detach()).float().T.unsqueeze(2)
/tmp/ipykernel_25497/2295200637.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(VOUT.clone().detach()).float().T


(torch.Size([63993, 1507, 1]), torch.Size([63993, 1507]))

- Randomly shuffle all simulation traces.
- Train, Validation, Test Split

In [22]:
# For small dataset

rand_idx = torch.randperm(X.shape[0])

toss_idx = rand_idx[:int(X.shape[0]*0.5)]
train_idx =rand_idx[int(X.shape[0]*0.5):int(X.shape[0]*0.7)]
valid_idx = rand_idx[int(X.shape[0]*0.8):int(X.shape[0]*0.95)]
test_idx  = rand_idx[int(X.shape[0]*0.95):]

In [22]:
# For large dataset
rand_idx = torch.randperm(X.shape[0])

train_idx = rand_idx[:int(X.shape[0]*0.8)]
valid_idx = rand_idx[int(X.shape[0]*0.8):int(X.shape[0]*0.9)]
test_idx  = rand_idx[int(X.shape[0]*0.9):]

In [23]:
X_train = X[train_idx]
Y_train = Y[train_idx]
X_valid = X[valid_idx]
Y_valid = Y[valid_idx]
X_test  = X[test_idx]
Y_test  = Y[test_idx]

data = {'X_train':X_train, 'Y_train':Y_train,
        'X_valid':X_valid, 'Y_valid':Y_valid,
        'X_test':X_test, 'Y_test':Y_test}

- Normalization + downsampling
- This is a form of dimensionality reduction so the model doesn’t have to process the full 1507-point signal

In [24]:
# :1500:15 means: take columns 0, 15, 30, …, 1495.
# That gives you 100 evenly spaced points across the whole 1500-step window

for k,v in data.items():
    norms = [1.0e-04, 1.0e-04, 1.0e-04, 1.0e-04, 50000, 0.001]
    if "X" in k:
        X1 = v[:,:1500:15,:]
        X2 = v[:,1501:,:]
        for i in range(6):
            X2[:,i,:] /= norms[i]
        X = torch.cat((X1, X2), dim=1)
    else:
        X = v[:,:1500:15]

    data[k] = X

# Simply take the first 100 time steps in full resolution

for k,v in data.items():
    norms = [1.0e-04, 1.0e-04, 1.0e-04, 1.0e-04, 50000, 0.001]
    if "X" in k:
        X1 = v[:,:100,:]
        X2 = v[:,1501:,:]
        for i in range(6):
            X2[:,i,:] /= norms[i]
        X = torch.cat((X1, X2), dim=1)
    else:
        X = v[:,:100]

    data[k] = X

In [25]:
# Check the size for our datasets.
for name, subset in data.items():
    print(name, subset.shape)

X_train torch.Size([12799, 106, 1])
Y_train torch.Size([12799, 100])
X_valid torch.Size([9599, 106, 1])
Y_valid torch.Size([9599, 100])
X_test torch.Size([3200, 106, 1])
Y_test torch.Size([3200, 100])


In [26]:
# Save dataset
# torch.save(data, f'./data/dataset.ds')

# or 

torch.save(data, f'./data/small_dataset.ds')

# 5. Second Dataset: Flexible Neuromorphic Circuit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import re
import warnings
import polars as pl

In [1]:
DATA_LOCATION = "data/V3/"

In [4]:
df1 = pl.read_csv(str(DATA_LOCATION + "Power_const_filtered.csv"))
df2 = pl.read_csv(str(DATA_LOCATION + "Power_Sin_filtered.csv"))
df3 = pl.read_csv(str(DATA_LOCATION + "VIN_const_filtered.csv"))
df4 = pl.read_csv(str(DATA_LOCATION + "VIN_Sin_filtered.csv"))
df5 = pl.read_csv(str(DATA_LOCATION + "Vout_const_filtered.csv"))
df6 = pl.read_csv(str(DATA_LOCATION + "Vout_Sin_filtered.csv"))

In [12]:
print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)
print(df6.shape)

(10001, 10001)
(10001, 7129)
(10001, 10001)
(10001, 7129)
(10001, 10001)
(10001, 7045)


In [5]:
df3.head()

/VIN (VIN=0) X,/VIN (VIN=0) Y,/VIN (VIN=0.00030003) Y,/VIN (VIN=0.00060006) Y,/VIN (VIN=0.00090009) Y,/VIN (VIN=0.00120012) Y,/VIN (VIN=0.00150015) Y,/VIN (VIN=0.00180018) Y,/VIN (VIN=0.00210021) Y,/VIN (VIN=0.00240024) Y,/VIN (VIN=0.00270027) Y,/VIN (VIN=0.0030003) Y,/VIN (VIN=0.00330033) Y,/VIN (VIN=0.00360036) Y,/VIN (VIN=0.00390039) Y,/VIN (VIN=0.00420042) Y,/VIN (VIN=0.00450045) Y,/VIN (VIN=0.00480048) Y,/VIN (VIN=0.00510051) Y,/VIN (VIN=0.00540054) Y,/VIN (VIN=0.00570057) Y,/VIN (VIN=0.0060006) Y,/VIN (VIN=0.00630063) Y,/VIN (VIN=0.00660066) Y,/VIN (VIN=0.00690069) Y,/VIN (VIN=0.00720072) Y,/VIN (VIN=0.00750075) Y,/VIN (VIN=0.00780078) Y,/VIN (VIN=0.00810081) Y,/VIN (VIN=0.00840084) Y,/VIN (VIN=0.00870087) Y,/VIN (VIN=0.0090009) Y,/VIN (VIN=0.00930093) Y,/VIN (VIN=0.00960096) Y,/VIN (VIN=0.00990099) Y,/VIN (VIN=0.01020102) Y,/VIN (VIN=0.01050105) Y,…,/VIN (VIN=2.989499) Y,/VIN (VIN=2.989799) Y,/VIN (VIN=2.990099) Y,/VIN (VIN=2.990399) Y,/VIN (VIN=2.990699) Y,/VIN (VIN=2.990999) Y,/VIN (VIN=2.991299) Y,/VIN (VIN=2.991599) Y,/VIN (VIN=2.991899) Y,/VIN (VIN=2.992199) Y,/VIN (VIN=2.992499) Y,/VIN (VIN=2.992799) Y,/VIN (VIN=2.993099) Y,/VIN (VIN=2.993399) Y,/VIN (VIN=2.993699) Y,/VIN (VIN=2.993999) Y,/VIN (VIN=2.994299) Y,/VIN (VIN=2.994599) Y,/VIN (VIN=2.994899) Y,/VIN (VIN=2.9952) Y,/VIN (VIN=2.9955) Y,/VIN (VIN=2.9958) Y,/VIN (VIN=2.9961) Y,/VIN (VIN=2.9964) Y,/VIN (VIN=2.9967) Y,/VIN (VIN=2.997) Y,/VIN (VIN=2.9973) Y,/VIN (VIN=2.9976) Y,/VIN (VIN=2.9979) Y,/VIN (VIN=2.9982) Y,/VIN (VIN=2.9985) Y,/VIN (VIN=2.9988) Y,/VIN (VIN=2.9991) Y,/VIN (VIN=2.9994) Y,/VIN (VIN=2.9997) Y,/VIN (VIN=3) Y,/VIN (VIN=1.79808) Y
f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64
0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
0.001,0,0.0003,0.0006,0.0009,0.0012,0.0015,0.0018,0.0021,0.0024,0.0027,0.003,0.0033,0.0036,0.0039,0.0042,0.0045,0.0048,0.005101,0.005401,0.005701,0.006001,0.006301,0.006601,0.006901,0.007201,0.007501,0.007801,0.008101,0.008401,0.008701,0.009001,0.009301,0.009601,0.009901,0.010201,0.010501,…,2.989499,2.989799,2.990099,2.990399,2.990699,2.990999,2.991299,2.991599,2.991899,2.992199,2.992499,2.992799,2.993099,2.993399,2.993699,2.993999,2.994299,2.994599,2.994899,2.9952,2.9955,2.9958,2.9961,2.9964,2.9967,2.997,2.9973,2.9976,2.9979,2.9982,2.9985,2.9988,2.9991,2.9994,2.9997,3,1.79808
0.002,0,0.0003,0.0006,0.0009,0.0012,0.0015,0.0018,0.0021,0.0024,0.0027,0.003,0.0033,0.0036,0.0039,0.0042,0.0045,0.0048,0.005101,0.005401,0.005701,0.006001,0.006301,0.006601,0.006901,0.007201,0.007501,0.007801,0.008101,0.008401,0.008701,0.009001,0.009301,0.009601,0.009901,0.010201,0.010501,…,2.989499,2.989799,2.990099,2.990399,2.990699,2.990999,2.991299,2.991599,2.991899,2.992199,2.992499,2.992799,2.993099,2.993399,2.993699,2.993999,2.994299,2.994599,2.994899,2.9952,2.9955,2.9958,2.9961,2.9964,2.9967,2.997,2.9973,2.9976,2.9979,2.9982,2.9985,2.9988,2.9991,2.9994,2.9997,3,1.79808
0.003,0,0.0003,0.0006,0.0009,0.0012,0.0015,0.0018,0.0021,0.0024,0.0027,0.003,0.0033,0.0036,0.0039,0.0042,0.0045,0.0048,0.005101,0.005401,0.005701,0.006001,0.006301,0.006601,0.006901,0.007201,0.007501,0.007801,0.008101,0.008401,0.008701,0.009001,0.009301,0.009601,0.009901,0.010201,0.010501,…,2.989499,2.989799,2.990099,2.990399,2.990699,2.990999,2.991299,2.991599,2.991899,2.992199,2.992499,2.992799,2.993099,2.993399,2.993699,2.993999,2.994299,2.994599,2.994899,2.9952,2.9955,2.9958,2.9961,2.9964,2.9967,2.997,2.9973,2.9976,2.9979,2.9982,2.9985,2.9988,2.9991,2.9994,2.9997,3,1.79808
0.004,0,0.0003

In [6]:
df4.head()

"/VIN (a=0.5,b=1,f=1) X","/VIN (a=0.5,b=1,f=1) Y","/VIN (a=0.5,b=1,f=1.5) Y","/VIN (a=0.5,b=1,f=2) Y","/VIN (a=0.5,b=1,f=2.5) Y","/VIN (a=0.5,b=1,f=3) Y","/VIN (a=0.5,b=1,f=3.5) Y","/VIN (a=0.5,b=1,f=4) Y","/VIN (a=0.5,b=1,f=4.5) Y","/VIN (a=0.5,b=1,f=5) Y","/VIN (a=0.5,b=6,f=1) Y","/VIN (a=0.5,b=6,f=1.5) Y","/VIN (a=0.5,b=6,f=2) Y","/VIN (a=0.5,b=6,f=2.5) Y","/VIN (a=0.5,b=6,f=3) Y","/VIN (a=0.5,b=6,f=3.5) Y","/VIN (a=0.5,b=6,f=4) Y","/VIN (a=0.5,b=6,f=4.5) Y","/VIN (a=0.5,b=6,f=5) Y","/VIN (a=0.5,b=11,f=1) Y","/VIN (a=0.5,b=11,f=1.5) Y","/VIN (a=0.5,b=11,f=2) Y","/VIN (a=0.5,b=11,f=2.5) Y","/VIN (a=0.5,b=11,f=3) Y","/VIN (a=0.5,b=11,f=3.5) Y","/VIN (a=0.5,b=11,f=4) Y","/VIN (a=0.5,b=11,f=4.5) Y","/VIN (a=0.5,b=11,f=5) Y","/VIN (a=0.5,b=16,f=1) Y","/VIN (a=0.5,b=16,f=1.5) Y","/VIN (a=0.5,b=16,f=2) Y","/VIN (a=0.5,b=16,f=2.5) Y","/VIN (a=0.5,b=16,f=3) Y","/VIN (a=0.5,b=16,f=3.5) Y","/VIN (a=0.5,b=16,f=4) Y","/VIN (a=0.5,b=16,f=4.5) Y","/VIN (a=0.5,b=16,f=5) Y",…,"/VIN (a=1.5,b=341,f=1) Y","/VIN (a=1.5,b=341,f=1.5) Y","/VIN (a=1.5,b=341,f=2) Y","/VIN (a=1.5,b=341,f=2.5) Y","/VIN (a=1.5,b=341,f=3) Y","/VIN (a=1.5,b=341,f=3.5) Y","/VIN (a=1.5,b=341,f=4) Y","/VIN (a=1.5,b=341,f=4.5) Y","/VIN (a=1.5,b=341,f=5) Y","/VIN (a=1.5,b=346,f=1) Y","/VIN (a=1.5,b=346,f=1.5) Y","/VIN (a=1.5,b=346,f=2) Y","/VIN (a=1.5,b=346,f=2.5) Y","/VIN (a=1.5,b=346,f=3) Y","/VIN (a=1.5,b=346,f=3.5) Y","/VIN (a=1.5,b=346,f=4) Y","/VIN (a=1.5,b=346,f=4.5) Y","/VIN (a=1.5,b=346,f=5) Y","/VIN (a=1.5,b=351,f=1) Y","/VIN (a=1.5,b=351,f=1.5) Y","/VIN (a=1.5,b=351,f=2) Y","/VIN (a=1.5,b=351,f=2.5) Y","/VIN (a=1.5,b=351,f=3) Y","/VIN (a=1.5,b=351,f=3.5) Y","/VIN (a=1.5,b=351,f=4) Y","/VIN (a=1.5,b=351,f=4.5) Y","/VIN (a=1.5,b=351,f=5) Y","/VIN (a=1.5,b=356,f=1) Y","/VIN (a=1.5,b=356,f=1.5) Y","/VIN (a=1.5,b=356,f=2) Y","/VIN (a=1.5,b=356,f=2.5) Y","/VIN (a=1.5,b=356,f=3) Y","/VIN (a=1.5,b=356,f=3.5) Y","/VIN (a=1.5,b=356,f=4) Y","/VIN (a=1.5,b=356,f=4.5) Y","/VIN (a=1.5,b=356,f=5) Y","/VIN (a=1.2,b=276,f=2.5) Y"
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.508726,0.508726,0.508726,0.508726,0.508726,0.508726,0.508726,0.508726,0.508726,0.552264,0.552264,0.552264,0.552264,0.552264,0.552264,0.552264,0.552264,0.552264,0.595404,0.595404,0.595404,0.595404,0.595404,0.595404,0.595404,0.595404,0.595404,0.637819,0.637819,0.637819,0.637819,0.637819,0.637819,0.637819,0.637819,0.637819,…,1.011648,1.011648,1.011648,1.011648,1.011648,1.011648,1.011648,1.011648,1.011648,1.137117,1.137117,1.137117,1.137117,1.137117,1.137117,1.137117,1.137117,1.137117,1.265348,1.265348,1.265348,1.265348,1.265348,1.265348,1.265348,1.265348,1.265348,1.395365,1.395365,1.395365,1.395365,1.395365,1.395365,1.395365,1.395365,1.395365,0.006574
0.001,0.511867,0.513437,0.515007,0.516577,0.518147,0.519717,0.521286,0.522855,0.524423,0.555387,0.556948,0.558508,0.560068,0.561627,0.563185,0.564742,0.566298,0.567855,0.598486,0.600025,0.601564,0.603101,0.604636,0.606171,0.607704,0.609237,0.610767,0.640835,0.642342,0.643846,0.645349,0.64685,0.64835,0.649848,0.651344,0.652839,…,1.02057,1.025039,1.029513,1.033992,1.038477,1.042966,1.047461,1.05196,1.056464,1.14627,1.150852,1.155438,1.160028,1.164622,1.169219,1.17382,1.178424,1.183032,1.274662,1.279323,1.283986,1.288651,1.293319,1.297989,1.302661,1.307335,1.312011,1.404769,1.409473,1.414177,1.418883,1.423589,1.428296,1.433004,1.437712,1.442421,0.008709
0.002,0.515008,0.518147,0.521286,0.524424,0.527561,0.530697,0.533831,0.536965,0.540095,0.558508,0.561627,0.564743,0.567856,0.570966,0.574074,0.577178,0.580278,0.583376,0.601564,0.604638,0.607707,0.610771,0.613832,0.616887,0.619937,0.622982,0.626022,0.643847,0.646852,0.649851,0.652844,0.65583,0.65881,0.661783,0.664749,0.667708,…,1.02951,1.038471,1.04745,1.056448,1.0654

In [7]:
def safe_concat(dfs, how="horizontal"):
    """
    Concatenate Polars DataFrames horizontally,
    automatically renaming duplicate columns (X, X_2, X_3, ...).
    """
    seen = {}
    renamed = []
    
    for df in dfs:
        new_cols = []
        for c in df.columns:
            if c not in seen:
                seen[c] = 1
                new_cols.append(c)
            else:
                seen[c] += 1
                new_cols.append(f"{c}_{seen[c]}")
        renamed.append(df.rename(dict(zip(df.columns, new_cols))))
    
    return pl.concat(renamed, how=how)

In [9]:
%%time
df_input_concat = safe_concat([df3, df4])
df_input_concat.head()

CPU times: user 42.4 ms, sys: 1.97 ms, total: 44.3 ms
Wall time: 42.5 ms


/VIN (VIN=0) X,/VIN (VIN=0) Y,/VIN (VIN=0.00030003) Y,/VIN (VIN=0.00060006) Y,/VIN (VIN=0.00090009) Y,/VIN (VIN=0.00120012) Y,/VIN (VIN=0.00150015) Y,/VIN (VIN=0.00180018) Y,/VIN (VIN=0.00210021) Y,/VIN (VIN=0.00240024) Y,/VIN (VIN=0.00270027) Y,/VIN (VIN=0.0030003) Y,/VIN (VIN=0.00330033) Y,/VIN (VIN=0.00360036) Y,/VIN (VIN=0.00390039) Y,/VIN (VIN=0.00420042) Y,/VIN (VIN=0.00450045) Y,/VIN (VIN=0.00480048) Y,/VIN (VIN=0.00510051) Y,/VIN (VIN=0.00540054) Y,/VIN (VIN=0.00570057) Y,/VIN (VIN=0.0060006) Y,/VIN (VIN=0.00630063) Y,/VIN (VIN=0.00660066) Y,/VIN (VIN=0.00690069) Y,/VIN (VIN=0.00720072) Y,/VIN (VIN=0.00750075) Y,/VIN (VIN=0.00780078) Y,/VIN (VIN=0.00810081) Y,/VIN (VIN=0.00840084) Y,/VIN (VIN=0.00870087) Y,/VIN (VIN=0.0090009) Y,/VIN (VIN=0.00930093) Y,/VIN (VIN=0.00960096) Y,/VIN (VIN=0.00990099) Y,/VIN (VIN=0.01020102) Y,/VIN (VIN=0.01050105) Y,…,"/VIN (a=1.5,b=341,f=1) Y","/VIN (a=1.5,b=341,f=1.5) Y","/VIN (a=1.5,b=341,f=2) Y","/VIN (a=1.5,b=341,f=2.5) Y","/VIN (a=1.5,b=341,f=3) Y","/VIN (a=1.5,b=341,f=3.5) Y","/VIN (a=1.5,b=341,f=4) Y","/VIN (a=1.5,b=341,f=4.5) Y","/VIN (a=1.5,b=341,f=5) Y","/VIN (a=1.5,b=346,f=1) Y","/VIN (a=1.5,b=346,f=1.5) Y","/VIN (a=1.5,b=346,f=2) Y","/VIN (a=1.5,b=346,f=2.5) Y","/VIN (a=1.5,b=346,f=3) Y","/VIN (a=1.5,b=346,f=3.5) Y","/VIN (a=1.5,b=346,f=4) Y","/VIN (a=1.5,b=346,f=4.5) Y","/VIN (a=1.5,b=346,f=5) Y","/VIN (a=1.5,b=351,f=1) Y","/VIN (a=1.5,b=351,f=1.5) Y","/VIN (a=1.5,b=351,f=2) Y","/VIN (a=1.5,b=351,f=2.5) Y","/VIN (a=1.5,b=351,f=3) Y","/VIN (a=1.5,b=351,f=3.5) Y","/VIN (a=1.5,b=351,f=4) Y","/VIN (a=1.5,b=351,f=4.5) Y","/VIN (a=1.5,b=351,f=5) Y","/VIN (a=1.5,b=356,f=1) Y","/VIN (a=1.5,b=356,f=1.5) Y","/VIN (a=1.5,b=356,f=2) Y","/VIN (a=1.5,b=356,f=2.5) Y","/VIN (a=1.5,b=356,f=3) Y","/VIN (a=1.5,b=356,f=3.5) Y","/VIN (a=1.5,b=356,f=4) Y","/VIN (a=1.5,b=356,f=4.5) Y","/VIN (a=1.5,b=356,f=5) Y","/VIN (a=1.2,b=276,f=2.5) Y"
f64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,1.011648,1.011648,1.011648,1.011648,1.011648,1.011648,1.011648,1.011648,1.011648,1.137117,1.137117,1.137117,1.137117,1.137117,1.137117,1.137117,1.137117,1.137117,1.265348,1.265348,1.265348,1.265348,1.265348,1.265348,1.265348,1.265348,1.265348,1.395365,1.395365,1.395365,1.395365,1.395365,1.395365,1.395365,1.395365,1.395365,0.006574
0.001,0,0.0003,0.0006,0.0009,0.0012,0.0015,0.0018,0.0021,0.0024,0.0027,0.003,0.0033,0.0036,0.0039,0.0042,0.0045,0.0048,0.005101,0.005401,0.005701,0.006001,0.006301,0.006601,0.006901,0.007201,0.007501,0.007801,0.008101,0.008401,0.008701,0.009001,0.009301,0.009601,0.009901,0.010201,0.010501,…,1.02057,1.025039,1.029513,1.033992,1.038477,1.042966,1.047461,1.05196,1.056464,1.14627,1.150852,1.155438,1.160028,1.164622,1.169219,1.17382,1.178424,1.183032,1.274662,1.279323,1.283986,1.288651,1.293319,1.297989,1.302661,1.307335,1.312011,1.404769,1.409473,1.414177,1.418883,1.423589,1.428296,1.433004,1.437712,1.442421,0.008709
0.002,0,0.0003,0.0006,0.0009,0.0012,0.0015,0.0018,0.0021,0.0024,0.0027,0.003,0.0033,0.0036,0.0039,0.0042,0.0045,0.0048,0.005101,0.005401,0.005701,0.006001,0.006301,0.006601,0.006901,0.007201,0.007501,0.007801,0.008101,0.008401,0.008701,0.009001,0.009301,0.009601,0.009901,0.010201,0.010501,…,1.02951,1.038471,1.04745,1.056448,1.065465,1.074499,1.08355,1.092619,1.101704,1.155436,1.164618,1.173813,1.183021,1.192242,1.201476,1.210722,1.21998,1.229249,1.283985,1.293317,1.302657,1.312005,1.321361,1.330724,1.340094,1.34947,1.358852,1.414177,1.423588,1.433003,1.44242,1.451839,1.46126,1.470682,1.480105,1.489529,0.011132
0.003,0,0.0003,0.0006,0.0009,0.0012,0.0015,0.0018,0.0021,0.0024,0.0027,0.003,0.0

In [10]:
%%time
df_output_concat = safe_concat([df5, df6])
df_output_concat.head()

CPU times: user 57 ms, sys: 6.17 ms, total: 63.1 ms
Wall time: 61.4 ms


/Vout_buffer2 (VIN=0) X,/Vout_buffer2 (VIN=0) Y,/Vout_buffer2 (VIN=0.00030003) Y,/Vout_buffer2 (VIN=0.00060006) Y,/Vout_buffer2 (VIN=0.00090009) Y,/Vout_buffer2 (VIN=0.00120012) Y,/Vout_buffer2 (VIN=0.00150015) Y,/Vout_buffer2 (VIN=0.00180018) Y,/Vout_buffer2 (VIN=0.00210021) Y,/Vout_buffer2 (VIN=0.00240024) Y,/Vout_buffer2 (VIN=0.00270027) Y,/Vout_buffer2 (VIN=0.0030003) Y,/Vout_buffer2 (VIN=0.00330033) Y,/Vout_buffer2 (VIN=0.00360036) Y,/Vout_buffer2 (VIN=0.00390039) Y,/Vout_buffer2 (VIN=0.00420042) Y,/Vout_buffer2 (VIN=0.00450045) Y,/Vout_buffer2 (VIN=0.00480048) Y,/Vout_buffer2 (VIN=0.00510051) Y,/Vout_buffer2 (VIN=0.00540054) Y,/Vout_buffer2 (VIN=0.00570057) Y,/Vout_buffer2 (VIN=0.0060006) Y,/Vout_buffer2 (VIN=0.00630063) Y,/Vout_buffer2 (VIN=0.00660066) Y,/Vout_buffer2 (VIN=0.00690069) Y,/Vout_buffer2 (VIN=0.00720072) Y,/Vout_buffer2 (VIN=0.00750075) Y,/Vout_buffer2 (VIN=0.00780078) Y,/Vout_buffer2 (VIN=0.00810081) Y,/Vout_buffer2 (VIN=0.00840084) Y,/Vout_buffer2 (VIN=0.00870087) Y,/Vout_buffer2 (VIN=0.0090009) Y,/Vout_buffer2 (VIN=0.00930093) Y,/Vout_buffer2 (VIN=0.00960096) Y,/Vout_buffer2 (VIN=0.00990099) Y,/Vout_buffer2 (VIN=0.01020102) Y,/Vout_buffer2 (VIN=0.01050105) Y,…,"/Vout_buffer2 (a=1.5,b=291,f=4) Y","/Vout_buffer2 (a=1.5,b=291,f=4.5) Y","/Vout_buffer2 (a=1.5,b=291,f=5) Y","/Vout_buffer2 (a=1.5,b=296,f=1) Y","/Vout_buffer2 (a=1.5,b=296,f=1.5) Y","/Vout_buffer2 (a=1.5,b=296,f=2) Y","/Vout_buffer2 (a=1.5,b=296,f=2.5) Y","/Vout_buffer2 (a=1.5,b=296,f=3) Y","/Vout_buffer2 (a=1.5,b=296,f=3.5) Y","/Vout_buffer2 (a=1.5,b=296,f=4) Y","/Vout_buffer2 (a=1.5,b=296,f=4.5) Y","/Vout_buffer2 (a=1.5,b=296,f=5) Y","/Vout_buffer2 (a=1.5,b=301,f=1) Y","/Vout_buffer2 (a=1.5,b=301,f=1.5) Y","/Vout_buffer2 (a=1.5,b=301,f=2) Y","/Vout_buffer2 (a=1.5,b=301,f=2.5) Y","/Vout_buffer2 (a=1.5,b=301,f=3) Y","/Vout_buffer2 (a=1.5,b=301,f=3.5) Y","/Vout_buffer2 (a=1.5,b=301,f=4) Y","/Vout_buffer2 (a=1.5,b=301,f=4.5) Y","/Vout_buffer2 (a=1.5,b=301,f=5) Y","/Vout_buffer2 (a=1.5,b=306,f=1) Y","/Vout_buffer2 (a=1.5,b=306,f=1.5) Y","/Vout_buffer2 (a=1.5,b=306,f=2) Y","/Vout_buffer2 (a=1.5,b=306,f=2.5) Y","/Vout_buffer2 (a=1.5,b=306,f=3) Y","/Vout_buffer2 (a=1.5,b=306,f=3.5) Y","/Vout_buffer2 (a=1.5,b=306,f=4) Y","/Vout_buffer2 (a=1.5,b=306,f=4.5) Y","/Vout_buffer2 (a=1.5,b=306,f=5) Y","/Vout_buffer2 (a=1.5,b=311,f=1) Y","/Vout_buffer2 (a=1.5,b=311,f=1.5) Y","/Vout_buffer2 (a=1.5,b=311,f=2) Y","/Vout_buffer2 (a=1.5,b=311,f=2.5) Y","/Vout_buffer2 (a=1.5,b=311,f=3) Y","/Vout_buffer2 (a=1.5,b=311,f=3.5) Y","/Vout_buffer2 (a=1,b=46,f=2) Y"
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0.001,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,…,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568,0.288568
0.002,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569,0.288569

In [11]:
print(df_input_concat.shape)
print(df_output_concat.shape)

(10001, 17130)
(10001, 17046)
